## ZodFrames & ZodSequences: an introduction
This notebook aims to introduce the ZodFrames & ZodSequences classes, which are helper classes to interact with the Frames and Sequences subsets of Zenseact Open Dataset (ZOD). It will highlight some basic functionality that can be used to build dataloaders in various frameworks, for example PyTorch.

In [13]:
# Import the package
from zod import ZodFrames, ZodSequences
from zod.dataclasses.zod_dataclasses import LidarData
import zod.constants as constants

In [23]:
# Initialize the class with the dataset root and version, version can be "mini" or "full"
zod_frames = ZodFrames(dataset_root="/staging/dataset_donation/round_2", version="mini")
zod_sequences = ZodSequences(dataset_root="/staging/dataset_donation/round_2", version="mini")

Loading val sequences: 100%|██████████| 1/1 [00:00<00:00, 226.49it/s]


In [15]:
# Get the training and validation split
training_frames = zod_frames.get_split(constants.TRAIN)
validation_frames = zod_frames.get_split(constants.VAL)

# Print the number of training and validation frames
print(f"Number of training frames: {len(training_frames)}")
print(f"Number of validation frames: {len(validation_frames)}")

# Print the number of training and validation sequences
training_sequences = zod_sequences.get_split(constants.TRAIN)
validation_sequences = zod_sequences.get_split(constants.VAL)
print(f"Number of training sequences: {len(training_sequences)}")
print(f"Number of validation sequences: {len(validation_sequences)}")

# Print out the first 5 training frames
print("The 5 first training frames have the ids:", training_frames[:5])

# Show the first training sequence
print("The first training sequence has the id:", training_sequences[0])

Number of training frames: 10
Number of validation frames: 2
Number of training sequences: 1
Number of validation sequences: 1
The 5 first training frames have the ids: ['009158', '018591', '023996', '029229', '044953']
The first training sequence has the id: 000002


In [16]:
# We can also get a frame by its id
frame_from_id = zod_frames["009158"]

# We can also get it via the index
frame_from_idx = zod_frames[9158]

# The two frames should be the same
assert frame_from_id.info == frame_from_idx.info

In [17]:
# We can use the frame to get the ego-motion of the vehicle
ego_motion = frame_from_id.ego_motion
print(f"Acceleration: {ego_motion.accelerations.shape}")
print(f"Velocities: {ego_motion.velocities.shape}")
print(f"Poses: {ego_motion.poses.shape}")
print(f"Timestamps: {ego_motion.timestamps.shape}")

Acceleration: (22, 3)
Velocities: (22, 3)
Poses: (22, 4, 4)
Timestamps: (22,)


In [18]:
# Note that the ego-motion is a lightweight version of the oxts data
oxts = frame_from_id.oxts
print(f"Acceleration: {oxts.accelerations.shape}")
print(f"Velocities: {oxts.velocities.shape}")
print(f"Poses: {oxts.poses.shape}")
print(f"Timestamps: {oxts.timestamps.shape}")

Acceleration: (3616, 3)
Velocities: (3616, 3)
Poses: (3616, 4, 4)
Timestamps: (3616,)


Note that the ZodFrames class yield a `ZodFrame` which acts a cache for the lightweight data (e.g., ego-motion, calibration, and metadata), but also holds an `info` attribute. This in turn holds all the paths to more heavyweight data (e.g., images and point clouds).

In [19]:
# Get the lidar core-frame for this ZodFrame
lidar_core_frame = frame_from_id.info.get_keyframe_lidar_frame()
print(lidar_core_frame)

{
  "filepath": "/staging/dataset_donation/round_2/single_frames/009158/lidar_velodyne/009158_india_2020-11-21T09:44:55.416922Z.npy",
  "time": "2020-11-21T09:44:55.416922Z"
}


In [20]:
# Load the lidar data
pc = LidarData.from_npy(lidar_core_frame.filepath)
print(f"Points: {pc.points.shape}")  # x, y, z
print(f"Timestamps: {pc.timestamps.shape}")
print(f"Intensity: {pc.intensity.shape}")
print(f"Diode: {pc.diode_idx.shape}")

Points: (298290, 3)
Timestamps: (298290,)
Intensity: (298290,)
Diode: (298290,)


In [21]:
# We can also iterate over all point-clouds
lidar_frames = frame_from_id.info.get_lidar_frames()
for i, lidar_frame in enumerate(lidar_frames):
    pc = LidarData.from_npy(lidar_frame.filepath)
    print(f"Lidar frame # {i}")
    print(f"Points: {pc.points.shape}")  # x, y, z
    print("-" * 20)

Lidar frame # 0
Points: (297756, 3)
--------------------
Lidar frame # 1
Points: (298300, 3)
--------------------
Lidar frame # 2
Points: (298155, 3)
--------------------
Lidar frame # 3
Points: (298749, 3)
--------------------
Lidar frame # 4
Points: (298738, 3)
--------------------
Lidar frame # 5
Points: (298576, 3)
--------------------
Lidar frame # 6
Points: (298743, 3)
--------------------
Lidar frame # 7
Points: (298735, 3)
--------------------
Lidar frame # 8
Points: (298716, 3)
--------------------
Lidar frame # 9
Points: (298541, 3)
--------------------
Lidar frame # 10
Points: (298290, 3)
--------------------
Lidar frame # 11
Points: (298109, 3)
--------------------
Lidar frame # 12
Points: (297850, 3)
--------------------
Lidar frame # 13
Points: (297175, 3)
--------------------
Lidar frame # 14
Points: (297078, 3)
--------------------
Lidar frame # 15
Points: (296966, 3)
--------------------
Lidar frame # 16
Points: (296954, 3)
--------------------
Lidar frame # 17
Points:

In [22]:
# We can do the same for the sequences
seq = zod_sequences[training_sequences[0]]

# Get the lidar frames
print(f"Number of lidar frames: {len(seq.info.get_lidar_frames(lidar=constants.Lidar.velodyne))}")

# We can also get the original camera frames
print(
    f"Number of camera frames: {len(seq.info.get_camera_frames(anonymization=constants.Anonymization.original))}"
)

# Or see how long the sequence is
print(f"Timespan: {(seq.info.end_time - seq.info.start_time).total_seconds()}")

Number of lidar frames: 181
Number of camera frames: 201
Timespan: 20.000197
